# Summary of this file

This file presents a comparison of various machine learning algorithms fitted to different combinations of modalities for two main purposes:

1. To observe the performance differences between these models.
2. To examine how the performance varies within the same models when using different modalities.

NOTE:
For categorical features, most algorithms cannot handle them directly, so we use one-hot encoding.

Results:
1. CatBoost and LGBM consistently outperform other models; therefore, our focus should be on these two models.
2. Including more modalities definitely leads to improved performance.

In [178]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import LabelEncoder
import catboost
from catboost import CatBoostClassifier, CatBoostRegressor
from sklearn.metrics import classification_report, accuracy_score
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import label_binarize
from sklearn.metrics import mean_squared_error
from sklearn.metrics import cohen_kappa_score
import matplotlib.pyplot as plt
from functools import partial
import scipy as sp
from sklearn.metrics import confusion_matrix as sk_cmatrix
from sklearn.model_selection import KFold
from joblib import dump
import re

import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import time
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, StratifiedKFold

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
import lightgbm as lgb
import catboost
import optuna
import joblib
import shap
import pickle

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import cohen_kappa_score

In [179]:
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0
    def _kappa_loss(self, coef, X, y):
        preds = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4])
        return -cohen_kappa_score(y, preds, weights='quadratic')
    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X = X, y = y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')
    def predict(self, X, coef):
        preds = pd.cut(X, [-np.inf] + list(np.sort(coef)) + [np.inf], labels = [0, 1, 2, 3, 4])
        return preds
    def coefficients(self):
        return self.coef_['x']

In [180]:
# df = pd.read_csv(f'petfinder-adoption-prediction/train/train.csv')
# X = df.drop(columns=['AdoptionSpeed'])
# y = df['AdoptionSpeed']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)
# df['iftrain'] = 0
# df.loc[df.index.isin(X_train.index), 'iftrain'] = 1

In [181]:
# df.to_csv(f'petfinder-adoption-prediction/train/train_split.csv')
df = pd.read_csv(f'petfinder-adoption-prediction/train/train_split.csv')
df_test = pd.read_csv(f'petfinder-adoption-prediction/test/test.csv')

In [188]:
2999 / (11994 + 2999)

0.20002667911692124

In [182]:
sum(df["iftrain"] == 1)

11994

In [183]:
sum(df["iftrain"] == 0)

2999

In [128]:
def PreProcess(
    df_in, bert=True, beit=True, breed=True, txt=True, meta=True, senti=True, newcols=True,
    pca_bert=200, pca_breed=200
):
    # get normal files
    df = df_in.copy(deep=True) 
    name = 'train'
    # extra modals
    if beit:
        beit_emb = pd.read_csv(f'petfinder-adoption-prediction/{name}/beit_emb.csv')
        beit_emb = beit_emb.drop(columns=['Description', 'PhotoAmt'])
        df = pd.merge(df, beit_emb, on=['PetID'], how='left')
    if bert and pca_bert > 0: 
        bert_pca200_ALL = pd.read_csv('petfinder-adoption-prediction/train/bert_pca200_ALL.csv') # 100 enough
        bert_emb = bert_pca200_ALL[['PetID'] + [f'bert_pc_{i}' for i in range(1, pca_bert + 1)]] # PetID
        df = pd.merge(df, bert_emb, on=['PetID'], how='left')
    if breed and pca_breed > 0: 
        breed_pca200_ALL = pd.read_csv('petfinder-adoption-prediction/train/breed_pca200_ALL.csv') # 100 enough
        breed_emb = breed_pca200_ALL[['BreedID'] + [f'breed_pc_{i}' for i in range(1, pca_breed + 1)]] # BreedID
        df = pd.merge(df, breed_emb, left_on=['Breed1'], right_on=['BreedID'], how='left')
        df = df.drop(columns=['BreedID'])  
    if txt: 
        txt_emb = pd.read_csv(f'petfinder-adoption-prediction/{name}/txt_emb.csv').drop(columns=['Description', 'PhotoAmt'])
        df = pd.merge(df, txt_emb, on=['PetID'], how='left')
    if meta: 
        metadata_gr = pd.read_csv(f'petfinder-adoption-prediction/{name}/metadata_gr.csv')
        df = pd.merge(df, metadata_gr, on=['PetID'], how='left')
    if senti: 
        sentiment_gr = pd.read_csv(f'petfinder-adoption-prediction/{name}/sentiment_gr.csv')
        df = pd.merge(df, sentiment_gr, on=['PetID'], how='left')
    if newcols: 
        new_cols_ALL = pd.read_csv('petfinder-adoption-prediction/train/new_cols_ALL.csv') # PetID
        df = pd.merge(df, new_cols_ALL, on=['PetID'], how='left')
    # catagorical one-hot encoding
    non_numeric_columns = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2', 'Color3', 'State']
    df_encoded = pd.get_dummies(df, columns=non_numeric_columns)
    # make sure feature names are valid
    df_encoded = df_encoded.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    df_encoded = df_encoded.fillna(0)
    # split into train & test
    train_df, test_df = df_encoded[df_encoded['iftrain'] == 1], df_encoded[df_encoded['iftrain'] == 0]
    to_drop = ['AdoptionSpeed', 'Name', 'Description', 'PetID', 'RescuerID', 'iftrain']
    X_train = train_df.drop(columns=to_drop)
    Y_train = train_df['AdoptionSpeed']
    X_test = test_df.drop(columns=to_drop)
    Y_test = test_df['AdoptionSpeed']
    # return results
    return X_train, X_test, Y_train, Y_test

In [129]:
bert = False
beit = False
breed=False
txt=False
meta=False
senti=False
newcols=False
pca_bert=0
pca_breed=0
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)

In [130]:
import pandas as pd

def quick_check_regression(X_train, X_test, y_train, y_test):
    models = {
        "Linear Regression": LinearRegression(),
        "Ridge Regression": Ridge(),
        "Lasso Regression": Lasso(),
        "ElasticNet": ElasticNet(),
        "KNN Regressor": KNeighborsRegressor(),
        "Decision Tree Regressor": DecisionTreeRegressor(),
        "Random Forest Regressor": RandomForestRegressor(n_jobs=-1),
        "Extra Trees Regressor": ExtraTreesRegressor(n_jobs=-1),
        "Gradient Boosting Regressor": GradientBoostingRegressor(),
        "XGBoost Regressor": XGBRegressor(n_jobs=-1),
        "LGBM Regressor": LGBMRegressor(verbose=0, n_jobs=-1),
        "CatBoost Regressor": CatBoostRegressor(verbose=0, thread_count=-1)  
    }

    # Initialize an empty DataFrame to store results
    results_df = pd.DataFrame(columns=["Model", "Cohen's Kappa Score"])

    for name, model in models.items():
        # print(f"\n\n=========={name}==========")
        model.fit(X_train, y_train)
        preds = model.predict(X_train)
        optR = OptimizedRounder()
        optR.fit(preds, y_train.values)
        coefficients = optR.coefficients()
        pred_y = optR.predict(model.predict(X_test), coefficients)
        kappa_score = cohen_kappa_score(y_test, pred_y, weights='quadratic')
        # print(f"Testing Cohen's Kappa Score : {kappa_score}")
        result = pd.DataFrame([[name, kappa_score]], columns=["Model", "Cohen's Kappa Score"])
        results_df = pd.concat([results_df, result], ignore_index=True)

    return results_df

## basic

In [132]:
# basic
bert=False
beit=False
breed=False
txt=False
meta=False
senti=False
newcols=False
pca_bert=0
pca_breed=0
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.277196
1,Ridge Regression,0.269159
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.055538
5,Decision Tree Regressor,0.148767
6,Random Forest Regressor,0.325306
7,Extra Trees Regressor,0.264927
8,Gradient Boosting Regressor,0.347603
9,XGBoost Regressor,0.355530


In [131]:
# basic + bert
bert=True
beit=False
breed=False
txt=False
meta=False
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.270944
1,Ridge Regression,0.267900
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.023488
5,Decision Tree Regressor,0.159754
6,Random Forest Regressor,0.320873
7,Extra Trees Regressor,0.278716
8,Gradient Boosting Regressor,0.357302
9,XGBoost Regressor,0.316796


In [133]:
# basic + bert + breed
bert=True
beit=False
breed=True
txt=False
meta=False
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.281011
1,Ridge Regression,0.290046
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.033857
5,Decision Tree Regressor,0.156326
6,Random Forest Regressor,0.328280
7,Extra Trees Regressor,0.273946
8,Gradient Boosting Regressor,0.371061
9,XGBoost Regressor,0.347755


In [134]:
# basic + beit
bert=False
beit=True
breed=False
txt=False
meta=False
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.361293
1,Ridge Regression,0.357689
2,Lasso Regression,0.142128
3,ElasticNet,0.201105
4,KNN Regressor,0.068275
5,Decision Tree Regressor,0.136886
6,Random Forest Regressor,0.337047
7,Extra Trees Regressor,0.269699
8,Gradient Boosting Regressor,0.382746
9,XGBoost Regressor,0.340353


In [135]:
# basic + senti
bert=False
beit=False
breed=False
txt=False
meta=False
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.257605
1,Ridge Regression,0.264652
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.054035
5,Decision Tree Regressor,0.195109
6,Random Forest Regressor,0.353109
7,Extra Trees Regressor,0.287375
8,Gradient Boosting Regressor,0.350322
9,XGBoost Regressor,0.348067


In [136]:
# basic + meta
bert=False
beit=False
breed=False
txt=False
meta=True
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.270442
1,Ridge Regression,0.273252
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.054324
5,Decision Tree Regressor,0.175885
6,Random Forest Regressor,0.315130
7,Extra Trees Regressor,0.274428
8,Gradient Boosting Regressor,0.353646
9,XGBoost Regressor,0.333815


In [137]:
# basic + meta + senti
bert=False
beit=False
breed=False
txt=False
meta=True
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.283345
1,Ridge Regression,0.273563
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.055846
5,Decision Tree Regressor,0.171747
6,Random Forest Regressor,0.310952
7,Extra Trees Regressor,0.282432
8,Gradient Boosting Regressor,0.344579
9,XGBoost Regressor,0.347978


In [138]:
# basic + bert + breed + beit
bert=True
beit=True
breed=True
txt=False
meta=False
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.357387
1,Ridge Regression,0.357400
2,Lasso Regression,0.142128
3,ElasticNet,0.201105
4,KNN Regressor,0.071388
5,Decision Tree Regressor,0.176817
6,Random Forest Regressor,0.342399
7,Extra Trees Regressor,0.274587
8,Gradient Boosting Regressor,0.391999
9,XGBoost Regressor,0.327104


In [139]:
# basic + txt
bert=False
beit=False
breed=False
txt=True
meta=False
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.265003
1,Ridge Regression,0.271992
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.033160
5,Decision Tree Regressor,0.181245
6,Random Forest Regressor,0.353442
7,Extra Trees Regressor,0.282094
8,Gradient Boosting Regressor,0.355735
9,XGBoost Regressor,0.338025


In [140]:
# basic + meta + senti + txt
bert=False
beit=False
breed=False
txt=True
meta=True
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.285284
1,Ridge Regression,0.283991
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.034403
5,Decision Tree Regressor,0.171461
6,Random Forest Regressor,0.325800
7,Extra Trees Regressor,0.289720
8,Gradient Boosting Regressor,0.352688
9,XGBoost Regressor,0.360985


In [141]:
# basic + bert + breed + beit + meta + senti + txt
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.350608
1,Ridge Regression,0.353137
2,Lasso Regression,0.142128
3,ElasticNet,0.201105
4,KNN Regressor,0.079761
5,Decision Tree Regressor,0.168066
6,Random Forest Regressor,0.351983
7,Extra Trees Regressor,0.273084
8,Gradient Boosting Regressor,0.389915
9,XGBoost Regressor,0.344324


In [142]:
# basic + meta + txt
bert=False
beit=False
breed=False
txt=True
meta=True
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.265555
1,Ridge Regression,0.272135
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.042851
5,Decision Tree Regressor,0.163290
6,Random Forest Regressor,0.325245
7,Extra Trees Regressor,0.274051
8,Gradient Boosting Regressor,0.343874
9,XGBoost Regressor,0.333087


In [143]:
# basic + bert + breed + beit + meta + txt
bert=True
beit=True
breed=True
txt=True
meta=True
senti=False
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.353788
1,Ridge Regression,0.352554
2,Lasso Regression,0.142128
3,ElasticNet,0.201105
4,KNN Regressor,0.069868
5,Decision Tree Regressor,0.157096
6,Random Forest Regressor,0.336815
7,Extra Trees Regressor,0.279979
8,Gradient Boosting Regressor,0.389915
9,XGBoost Regressor,0.336760


In [164]:
# basic + senti + txt
bert=False
beit=False
breed=False
txt=True
meta=False
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.279157
1,Ridge Regression,0.263087
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.030421
5,Decision Tree Regressor,0.178701
6,Random Forest Regressor,0.328698
7,Extra Trees Regressor,0.287769
8,Gradient Boosting Regressor,0.360281
9,XGBoost Regressor,0.340202


In [165]:
# basic + bert + breed + meta + senti + txt
bert=True
beit=False
breed=True
txt=True
meta=True
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.297927
1,Ridge Regression,0.287105
2,Lasso Regression,0.118243
3,ElasticNet,0.118513
4,KNN Regressor,0.037800
5,Decision Tree Regressor,0.160937
6,Random Forest Regressor,0.336515
7,Extra Trees Regressor,0.287242
8,Gradient Boosting Regressor,0.361581
9,XGBoost Regressor,0.315202


In [166]:
# basic + beit + meta + senti + txt
bert=False
beit=True
breed=False
txt=True
meta=True
senti=True
newcols=False
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.363571
1,Ridge Regression,0.361868
2,Lasso Regression,0.142128
3,ElasticNet,0.201105
4,KNN Regressor,0.074769
5,Decision Tree Regressor,0.160253
6,Random Forest Regressor,0.327660
7,Extra Trees Regressor,0.272656
8,Gradient Boosting Regressor,0.393159
9,XGBoost Regressor,0.339676


## advanced (always have newcols)

In [144]:
# advanced
bert=False
beit=False
breed=False
txt=False
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.273805
1,Ridge Regression,0.259892
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.036885
5,Decision Tree Regressor,0.206491
6,Random Forest Regressor,0.387661
7,Extra Trees Regressor,0.310489
8,Gradient Boosting Regressor,0.372062
9,XGBoost Regressor,0.389635


In [145]:
# advanced + bert
bert=True
beit=False
breed=False
txt=False
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.289077
1,Ridge Regression,0.296560
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.042856
5,Decision Tree Regressor,0.187694
6,Random Forest Regressor,0.340701
7,Extra Trees Regressor,0.292363
8,Gradient Boosting Regressor,0.378869
9,XGBoost Regressor,0.332810


In [146]:
# advanced + bert + breed
bert=True
beit=False
breed=True
txt=False
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.300786
1,Ridge Regression,0.295788
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.038995
5,Decision Tree Regressor,0.161794
6,Random Forest Regressor,0.363446
7,Extra Trees Regressor,0.302717
8,Gradient Boosting Regressor,0.384525
9,XGBoost Regressor,0.329816


In [147]:
# advanced + beit
bert=False
beit=True
breed=False
txt=False
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.360095
1,Ridge Regression,0.361321
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.098594
5,Decision Tree Regressor,0.158926
6,Random Forest Regressor,0.339911
7,Extra Trees Regressor,0.269012
8,Gradient Boosting Regressor,0.413167
9,XGBoost Regressor,0.364592


In [148]:
# advanced + senti
bert=False
beit=False
breed=False
txt=False
meta=False
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.261915
1,Ridge Regression,0.258894
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.037124
5,Decision Tree Regressor,0.232682
6,Random Forest Regressor,0.380010
7,Extra Trees Regressor,0.305477
8,Gradient Boosting Regressor,0.377414
9,XGBoost Regressor,0.391271


In [149]:
# advanced + meta
bert=False
beit=False
breed=False
txt=False
meta=True
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.272168
1,Ridge Regression,0.293908
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.044451
5,Decision Tree Regressor,0.222457
6,Random Forest Regressor,0.369446
7,Extra Trees Regressor,0.301832
8,Gradient Boosting Regressor,0.370663
9,XGBoost Regressor,0.365691


In [150]:
# advanced + meta + senti
bert=False
beit=False
breed=False
txt=False
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.265914
1,Ridge Regression,0.270127
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.032655
5,Decision Tree Regressor,0.220754
6,Random Forest Regressor,0.364021
7,Extra Trees Regressor,0.300124
8,Gradient Boosting Regressor,0.373594
9,XGBoost Regressor,0.371876


In [151]:
# advanced + bert + breed + beit
bert=True
beit=True
breed=True
txt=False
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.359556
1,Ridge Regression,0.359473
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.094857
5,Decision Tree Regressor,0.174447
6,Random Forest Regressor,0.349611
7,Extra Trees Regressor,0.267225
8,Gradient Boosting Regressor,0.408036
9,XGBoost Regressor,0.367402


In [152]:
# advanced + txt
bert=False
beit=False
breed=False
txt=True
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.259853
1,Ridge Regression,0.262853
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.046222
5,Decision Tree Regressor,0.214157
6,Random Forest Regressor,0.379197
7,Extra Trees Regressor,0.304902
8,Gradient Boosting Regressor,0.377144
9,XGBoost Regressor,0.381999


In [153]:
# advanced + meta + senti + txt
bert=False
beit=False
breed=False
txt=True
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.280032
1,Ridge Regression,0.295862
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.039069
5,Decision Tree Regressor,0.194573
6,Random Forest Regressor,0.352507
7,Extra Trees Regressor,0.307904
8,Gradient Boosting Regressor,0.372943
9,XGBoost Regressor,0.378099


In [154]:
# advanced + bert + breed + beit + meta + senti + txt (200)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.354415
1,Ridge Regression,0.353650
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.116632
5,Decision Tree Regressor,0.163672
6,Random Forest Regressor,0.353151
7,Extra Trees Regressor,0.275257
8,Gradient Boosting Regressor,0.406325
9,XGBoost Regressor,0.351019


In [155]:
# advanced + meta + txt
bert=False
beit=False
breed=False
txt=True
meta=True
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.284073
1,Ridge Regression,0.293760
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.043536
5,Decision Tree Regressor,0.231493
6,Random Forest Regressor,0.380616
7,Extra Trees Regressor,0.309221
8,Gradient Boosting Regressor,0.370272
9,XGBoost Regressor,0.365889


In [156]:
# advanced + bert + breed + beit + meta + txt
bert=True
beit=True
breed=True
txt=True
meta=True
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355962
1,Ridge Regression,0.357309
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.115449
5,Decision Tree Regressor,0.168348
6,Random Forest Regressor,0.356775
7,Extra Trees Regressor,0.282210
8,Gradient Boosting Regressor,0.406808
9,XGBoost Regressor,0.368986


In [157]:
# advanced + bert + breed + beit + meta + senti + txt (100)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=100
pca_breed=100
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355982
1,Ridge Regression,0.350168
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.104271
5,Decision Tree Regressor,0.211450
6,Random Forest Regressor,0.353174
7,Extra Trees Regressor,0.268891
8,Gradient Boosting Regressor,0.403664
9,XGBoost Regressor,0.351315


In [158]:
# advanced + bert + beit + meta + senti + txt (200)
bert=True
beit=True
breed=False
txt=True
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355066
1,Ridge Regression,0.359689
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.108251
5,Decision Tree Regressor,0.173265
6,Random Forest Regressor,0.344808
7,Extra Trees Regressor,0.284330
8,Gradient Boosting Regressor,0.410966
9,XGBoost Regressor,0.374258


In [159]:
# advanced + bert + breed + beit + meta + senti + txt (150)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=150
pca_breed=150
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.348822
1,Ridge Regression,0.347480
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.115532
5,Decision Tree Regressor,0.179753
6,Random Forest Regressor,0.352248
7,Extra Trees Regressor,0.277810
8,Gradient Boosting Regressor,0.410667
9,XGBoost Regressor,0.379341


In [160]:
# advanced + bert + breed + beit + meta + senti + txt (160)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=160
pca_breed=160
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.351723
1,Ridge Regression,0.356204
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.114663
5,Decision Tree Regressor,0.180240
6,Random Forest Regressor,0.335158
7,Extra Trees Regressor,0.275742
8,Gradient Boosting Regressor,0.407382
9,XGBoost Regressor,0.370754


In [161]:
# advanced + bert + breed + beit + meta + senti + txt (170)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=170
pca_breed=170
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.354052
1,Ridge Regression,0.357932
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.115482
5,Decision Tree Regressor,0.170850
6,Random Forest Regressor,0.351658
7,Extra Trees Regressor,0.281707
8,Gradient Boosting Regressor,0.405376
9,XGBoost Regressor,0.368754


In [162]:
# advanced + bert + breed + beit + meta + senti + txt (180)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=180
pca_breed=180
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355147
1,Ridge Regression,0.358992
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.115332
5,Decision Tree Regressor,0.179797
6,Random Forest Regressor,0.358298
7,Extra Trees Regressor,0.284969
8,Gradient Boosting Regressor,0.401535
9,XGBoost Regressor,0.374383


In [163]:
# advanced + bert + breed + beit + meta + senti + txt (190)
bert=True
beit=True
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=190
pca_breed=190
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355686
1,Ridge Regression,0.360686
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.116410
5,Decision Tree Regressor,0.167574
6,Random Forest Regressor,0.346525
7,Extra Trees Regressor,0.283856
8,Gradient Boosting Regressor,0.399107
9,XGBoost Regressor,0.357871


In [167]:
# advanced + senti + txt
bert=False
beit=False
breed=False
txt=True
meta=False
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.262647
1,Ridge Regression,0.274062
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.042841
5,Decision Tree Regressor,0.193685
6,Random Forest Regressor,0.358979
7,Extra Trees Regressor,0.308329
8,Gradient Boosting Regressor,0.370192
9,XGBoost Regressor,0.360477


In [168]:
# advanced + bert + breed + meta + senti + txt
bert=True
beit=False
breed=True
txt=True
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.301055
1,Ridge Regression,0.302560
2,Lasso Regression,0.126646
3,ElasticNet,0.118355
4,KNN Regressor,0.053614
5,Decision Tree Regressor,0.197386
6,Random Forest Regressor,0.343351
7,Extra Trees Regressor,0.299615
8,Gradient Boosting Regressor,0.380504
9,XGBoost Regressor,0.344906


In [169]:
# advanced + beit + meta + senti + txt
bert=False
beit=True
breed=False
txt=True
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.361229
1,Ridge Regression,0.366134
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.115534
5,Decision Tree Regressor,0.150603
6,Random Forest Regressor,0.348496
7,Extra Trees Regressor,0.275854
8,Gradient Boosting Regressor,0.411553
9,XGBoost Regressor,0.363515


In [170]:
# advanced + bert + beit
bert=True
beit=True
breed=False
txt=False
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.357255
1,Ridge Regression,0.356632
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.102372
5,Decision Tree Regressor,0.164472
6,Random Forest Regressor,0.338831
7,Extra Trees Regressor,0.275717
8,Gradient Boosting Regressor,0.413222
9,XGBoost Regressor,0.354306


## final compare

In [171]:
# advanced + T200 G s
bert=True
beit=True
breed=False
txt=False
meta=False
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.359588
1,Ridge Regression,0.366454
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.102898
5,Decision Tree Regressor,0.160201
6,Random Forest Regressor,0.348195
7,Extra Trees Regressor,0.289530
8,Gradient Boosting Regressor,0.413371
9,XGBoost Regressor,0.357263


In [172]:
# advanced + T200 b200 G s
bert=True
beit=True
breed=True
txt=False
meta=False
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.357693
1,Ridge Regression,0.361603
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.095363
5,Decision Tree Regressor,0.178152
6,Random Forest Regressor,0.350798
7,Extra Trees Regressor,0.291347
8,Gradient Boosting Regressor,0.407615
9,XGBoost Regressor,0.361920


In [173]:
# advanced + T200 G t
bert=True
beit=True
breed=False
txt=True
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.354620
1,Ridge Regression,0.357102
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.102238
5,Decision Tree Regressor,0.180147
6,Random Forest Regressor,0.345951
7,Extra Trees Regressor,0.280181
8,Gradient Boosting Regressor,0.412666
9,XGBoost Regressor,0.360892


In [174]:
# advanced + T200 b200 G t
bert=True
beit=True
breed=True
txt=True
meta=False
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.350615
1,Ridge Regression,0.360777
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.108986
5,Decision Tree Regressor,0.169858
6,Random Forest Regressor,0.350992
7,Extra Trees Regressor,0.281554
8,Gradient Boosting Regressor,0.402976
9,XGBoost Regressor,0.358714


In [175]:
# advanced + T200 G m t
bert=True
beit=True
breed=False
txt=True
meta=True
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.356593
1,Ridge Regression,0.366075
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.107044
5,Decision Tree Regressor,0.163289
6,Random Forest Regressor,0.344393
7,Extra Trees Regressor,0.281394
8,Gradient Boosting Regressor,0.411244
9,XGBoost Regressor,0.368325


In [176]:
# advanced + T200 b200 G m t
bert=True
beit=True
breed=True
txt=True
meta=True
senti=False
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355962
1,Ridge Regression,0.357309
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.115449
5,Decision Tree Regressor,0.179292
6,Random Forest Regressor,0.354176
7,Extra Trees Regressor,0.278495
8,Gradient Boosting Regressor,0.406429
9,XGBoost Regressor,0.368986


In [177]:
# advanced + T200 G m s t
bert=True
beit=True
breed=False
txt=True
meta=True
senti=True
newcols=True
pca_bert=200
pca_breed=200
X_train, X_test, y_train, y_test = PreProcess(
    df, bert, beit, breed, txt, meta, senti, newcols, pca_bert, pca_breed
)
result = quick_check_regression(X_train, X_test, y_train, y_test)
result

,Model,Cohen's Kappa Score
0,Linear Regression,0.355066
1,Ridge Regression,0.359689
2,Lasso Regression,0.149930
3,ElasticNet,0.159075
4,KNN Regressor,0.108251
5,Decision Tree Regressor,0.175303
6,Random Forest Regressor,0.348742
7,Extra Trees Regressor,0.283832
8,Gradient Boosting Regressor,0.410829
9,XGBoost Regressor,0.374258
